In [1]:
import re
from string import ascii_lowercase



def fetch_words(read_mode):
    '''Función no alterda por el ataque'''
    '''Agregué la opción `enconding="utf8"`'''

    words_from_dictionary = [ word.strip() for word in open('words.txt', encoding="utf8").readlines() ]
    words_from_books = re.findall(r'\w+', open('BOOKS.txt', read_mode, encoding="utf8").read())
    
    return words_from_dictionary + words_from_books

# El archivo se había abierto como escritura, pero se debe abrir en modo lectura.
WORDS = fetch_words('r') #<<<---
LETTERS = list(ascii_lowercase) + ['ñ', 'á', 'é', 'í', 'ó', 'ú']#+[',','?','¿']
SYMBOLS = ',.-¡!¿?#$@&%'

In [3]:
len(WORDS_INDEX)

80400

In [2]:
print(LETTERS)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ñ', 'á', 'é', 'í', 'ó', 'ú']


In [2]:
# Definimos un índice de palabras en el cual guardamos la frecuencia con la que se utiliza en nuestros
# textos de referencia
WORDS_INDEX = {}

# Las ordenes estaban mal y/o al reves.
for word in WORDS:
    if word in WORDS_INDEX:
        WORDS_INDEX.update({word:WORDS_INDEX[word]+ 1})
    else:
        WORDS_INDEX[word] = 1

for symbol in SYMBOLS:
    WORDS_INDEX[symbol]=1

In [4]:
def filter_real_words(words):
    return set(word for word in words if word.lower() in WORDS_INDEX) #Agregué el lowecase()

In [5]:
filter_real_words(".")

{'.'}

In [6]:
def one_length_edit(word):
    '''Función no alterda por el ataque'''
    
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    #print(splits)
    removals_of_one_letter = []
    
    for left, right in splits:
        if right:
            removals_of_one_letter.append(left + right[1:])
            
    two_letters_transposes = []
    
    for left, right in splits:
        if len(right) > 1:
            two_letters_transposes.append(left + right[1] + right[0] + right[2:])
            
    one_letter_replaces = []
    
    for left, right in splits:
        if right:
            for c in LETTERS:
                one_letter_replaces.append(left + c + right[1:])
                
    one_letter_insertions = []
    
    for left, right in splits:
        for c in LETTERS:
            one_letter_insertions.append(left + c + right)
    
    one_length_editions = removals_of_one_letter + two_letters_transposes + one_letter_replaces + one_letter_insertions
    
    return list(set(one_length_editions))



def two_lenght_edit(word):
    '''Función no alterda por el ataque'''
    return [e2 for e1 in one_length_edit(word) for e2 in one_length_edit(e1)]

In [7]:
def possible_corrections(word):
    cap=word[0].isupper()
    lword=word.lower()
    single_word_possible_corrections = filter_real_words([lword])
    one_length_edit_possible_corrections = filter_real_words(one_length_edit(lword))
    two_lenght_edit_possible_corrections = filter_real_words(two_lenght_edit(lword))
    no_correction_at_all = word
    ## Estaban trocados el primer y el tercer caso.
    if single_word_possible_corrections:
        if cap:
            return {option.title() for option in single_word_possible_corrections}
        return single_word_possible_corrections
    
    elif one_length_edit_possible_corrections:
        if cap:
            return {option.title() for option in one_length_edit_possible_corrections}
        return one_length_edit_possible_corrections
    
    elif two_lenght_edit_possible_corrections:
        if cap:
            return {option.title() for option in two_lenght_edit_possible_corrections}
        return two_lenght_edit_possible_corrections
    
    else:
        return no_correction_at_all

In [8]:
possible_corrections("Él")

{'Él'}

In [9]:
def spell_check_word(word):
    cap=word[0].isupper()
    if not cap:
        return max(possible_corrections(word.lower()), key=language_model)
    else:
        return max(possible_corrections(word.lower()), key=language_model).title()

In [10]:
def language_model(word):
    return WORDS_INDEX.get(word, 0)


In [11]:
def correct_split(sentence):
    """ Esta función crea una lista que divide la frase ubicando los signos aparte
        de las palabras
        
        Parametros
        ----------
        sentence: str
                La frase que se desea partir
    """
    stripped = list(map(lambda x : x, sentence.split()))
    stripped_sentence=[]
    for word in stripped:
        if any(car in word for car in ",.?¿¡!"):
            stripped_sentence.extend(word_split(word))
        else:
            stripped_sentence.append(word)
    return stripped_sentence

In [12]:
def word_split(word):
    punt=SYMBOLS
    if not any(p in word for p in punt):
        return [word]
    tw=word
    carry=[]
    while(len(punt)>0):
        if(tw.find(punt[0])!=-1):
            ri=tw[:tw.find(punt[0])]
            c=tw[tw.find(punt[0])]
            rd=tw[tw.find(punt[0])+1:]
            if(len(ri)!=0):
                carry.extend(word_split(ri))
            carry.append(c)
            if(len(rd)!=0):
                carry.extend(word_split(rd))
            break
        punt=punt[1:]
    return carry

In [13]:
def correct_join(words):
    """ Esta función crea la frase con el formato correcto de las palabras
    
        Parametro
        ---------
        words: list
             Las palabras que se desean conectar, puede contener signos de puntuación
    """
    sentence=""
    for i in range(len(words)):
        if i==0:
            if not any(car in words[i] for car in ",.?¿¡!"):
                sentence+=words[i]
        elif any(car in words[i] for car in ",.?¿¡!"):
            sentence+=words[i]
        elif len(sentence)==0:
            sentence+=words[i]
        else:
            sentence+=" "+words[i]
    return sentence

In [14]:
def spell_check_sentence(sentence):
    stripped_sentence = correct_split(sentence)#list(map(lambda x : x, lower_cased_sentence.split()))
    checked = map(spell_check_word, stripped_sentence) #<<-- Ojo que cambié ese filter por map
    checked_words=list(checked)
    return correct_join(checked_words)#' '.join(checked_words)

In [15]:
spell_check_sentence(".Él, no era una persona de fiar pues era un mentirozo")

'Él, no era una persona de fiar pues era un mentiroso'

In [16]:
def test_spell_check_sentence():

    sentence = 'fabor guardar cilencio para no molestar'
    assert 'favor guardar silencio para no molestar' == spell_check_sentence(sentence) 

    
    sentence = 'un lgar para la hopinion'
    #no ha lugar para la opinión
    assert 'un lugar para la opinión' == spell_check_sentence(sentence)

    sentence = 'el Arebol del día'
    print(spell_check_sentence(sentence))
    assert 'el arrebol del día' == spell_check_sentence(sentence)

    sentence = 'Rezpeto por la educasión'
    print(spell_check_sentence(sentence))
    assert 'respeto por la educación' == spell_check_sentence(sentence)

    sentence = 'RTe encanta conduzir'
    print(spell_check_sentence(sentence))
    assert 'te encanta conducir' == spell_check_sentence(sentence)

    sentence = 'HOy ay karne azada frezca siga pa dentro'
    print(spell_check_sentence(sentence))
    assert 'hoy ay carne azada fresca siga la dentro' == spell_check_sentence(sentence)

    sentence = 'En mi ezcuela no enseñan a escrivir ni a ler'
    print(spell_check_sentence(sentence))
    assert 'en mi escuela no enseñan a escribir ni a le' == spell_check_sentence(sentence)

    sentence = 'él no era una persona de fiar pues era un mentirozo'
    print(spell_check_sentence(sentence))
    assert 'él no era una persona de fiar pues era un mentiroso' == spell_check_sentence(sentence)
test_spell_check_sentence()

el Arrebol del día


AssertionError: 

In [ ]:
possible_corrections("%")

In [ ]:
WORDS_INDEX["atar"]

In [ ]:
WORDS_INDEX["matar"]

In [ ]:
possible_corrections("hatar")

---
---
# Solución 1


In [17]:
import re
from string import ascii_lowercase



def fetch_words(read_mode):
    '''Función no alterda por el ataque
       ---
       Se agregó la opción enconding="utf8" en la importación de cada archivo.
       De esta forma se evitan problemas de codificación.
    '''
    words_from_dictionary = [ word.strip() for word in open('words.txt', encoding="utf8").readlines() ]
    words_from_books = re.findall(r'\w+', open('BOOKS.txt', read_mode, encoding="utf8").read())
    return words_from_dictionary + words_from_books

# Abrimos el archivos en modo lectura y creamos el archivo de palabras y letras.
WORDS = fetch_words('r') 
LETTERS = list(ascii_lowercase) + ['ñ', 'á', 'é', 'í', 'ó', 'ú']

# Definimos un índice de palabras en el cual guardamos la frecuencia con la que se utiliza en nuestros
# textos de referencia. Esto nos será útil a la hora de "medir" la distancia entre una palabra mal
# escrita y un candidato.
WORDS_INDEX = {}
for word in WORDS:
    if word in WORDS_INDEX:
        WORDS_INDEX.update({word:WORDS_INDEX[word]+ 1}) # Si la palabra ya está aumentamos su frecuencia
    else:
        WORDS_INDEX[word] = 1 # Si la palabra no está la agregamos

def possible_corrections(word):
    ''' Busca las posibles correcciones que podemos hacer a una palabra
    
        El parámetro de búsqueda es el siguiente:
        1. Primero se revisa si la palabra pertenece al diccionario,
        2. Luego se miran las posibles palabras a distancia 1,
        3. Luego se miran las posibles palabras a distancia 2
        
        En caso tal que todo esto falle se devuelve la palabra sin cambio alguno
        
        Parámetro
        ---------
        word : str
             Palabra a consultar
    '''
    single_word_possible_corrections = filter_real_words([word])
    one_length_edit_possible_corrections = filter_real_words(one_length_edit(word))
    two_lenght_edit_possible_corrections = filter_real_words(two_lenght_edit(word))
    no_correction_at_all = word
    
    if single_word_possible_corrections:
        # Si la palabra pertenece al diccionario.
        return single_word_possible_corrections
    
    elif one_length_edit_possible_corrections:
        # Si hay correcciones de distancia 1
        return one_length_edit_possible_corrections
    
    elif two_lenght_edit_possible_corrections:
        # Si hay correcciones de diatancia 2
        return two_lenght_edit_possible_corrections
    
    else:
        # En cualquier otro caso se devuelve la palabra sin posibles correcciones
        return no_correction_at_all

def spell_check_sentence(sentence):
    ''' Busca las posibles correcciones que podemos hacer a una frase
        
        Parámetro
        ---------
        sentence : str
             Frase a corregir
    '''
    lower_cased_sentence = sentence.lower()
    stripped_sentence = list(map(lambda x : x.strip('.,?¿'), lower_cased_sentence.split()))
    checked = map(spell_check_word, stripped_sentence)
    checked_words=list(checked)
    return ' '.join(checked_words)

def spell_check_word(word):
    ''' Busca la mejor corrección que podemos hacer a una palabra
    
        En este caso el criterio que se sigue es el de más frecuencia
        
        Parámetro
        ---------
        word : str
             Palabra a consultar
    '''
    return max(possible_corrections(word), key=language_model)

def language_model(word):
    ''' Nos dice la frecuencia con la cual pasa una palabra candidata a ser la corrección
        
        Parámetro
        ---------
        word : str
             Palabra a consultar
    '''
    return WORDS_INDEX.get(word, 0)

def filter_real_words(words):
    ''' Busca determina cuales palabras de una lista son reales verificando si está en el
        diccionario
        
        Parámetro
        ---------
        words : list
             Palabras a consultar
    '''
    return set(word for word in words if word in WORDS_INDEX)

def one_length_edit(word):
    '''Función no alterda por el ataque'''
    
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    removals_of_one_letter = []
    
    for left, right in splits:
        if right:
            removals_of_one_letter.append(left + right[1:])
            
    two_letters_transposes = []
    
    for left, right in splits:
        if len(right) > 1:
            two_letters_transposes.append(left + right[1] + right[0] + right[2:])
            
    one_letter_replaces = []
    
    for left, right in splits:
        if right:
            for c in LETTERS:
                one_letter_replaces.append(left + c + right[1:])
                
    one_letter_insertions = []
    
    for left, right in splits:
        for c in LETTERS:
            one_letter_insertions.append(left + c + right)
    
    one_length_editions = removals_of_one_letter + two_letters_transposes + one_letter_replaces + one_letter_insertions
    
    return list(set(one_length_editions))



def two_lenght_edit(word):
    '''Función no alterda por el ataque'''
    return [e2 for e1 in one_length_edit(word) for e2 in one_length_edit(e1)]

def test_spell_check_sentence():

    sentence = 'fabor guardar cilencio para no molestar'
    assert 'favor guardar silencio para no molestar' == spell_check_sentence(sentence) 

    sentence = 'un lgar para la hopinion'
    assert 'un lugar para la opinión' == spell_check_sentence(sentence)

    sentence = 'el Arebol del día'
    print(spell_check_sentence(sentence))
    assert 'el arrebol del día' == spell_check_sentence(sentence)

    sentence = 'Rezpeto por la educasión'
    print(spell_check_sentence(sentence))
    assert 'respeto por la educación' == spell_check_sentence(sentence)

    sentence = 'RTe encanta conduzir'
    print(spell_check_sentence(sentence))
    assert 'te encanta conducir' == spell_check_sentence(sentence)

    sentence = 'HOy ay karne azada frezca siga pa dentro'
    print(spell_check_sentence(sentence))
    assert 'hoy ay carne azada fresca siga la dentro' == spell_check_sentence(sentence)

    sentence = 'En mi ezcuela no enseñan a escrivir ni a ler'
    print(spell_check_sentence(sentence))
    assert 'en mi escuela no enseñan a escribir ni a le' == spell_check_sentence(sentence)

    sentence = 'él no era una persona de fiar pues era un mentirozo'
    print(spell_check_sentence(sentence))
    assert 'él no era una persona de fiar pues era un mentiroso' == spell_check_sentence(sentence)
test_spell_check_sentence()

el arrebol del día
respeto por la educación
te encanta conducir
hoy ay carne azada fresca siga la dentro
en mi escuela no enseñan a escribir ni a le
él no era una persona de fiar pues era un mentiroso


---
---
# Solución 2

In [1]:
import re
from string import ascii_lowercase



def fetch_words(read_mode):
    '''Función no alterda por el ataque'''
    '''Agregué la opción `enconding="utf8"`'''

    words_from_dictionary = [ word.strip() for word in open('words.txt', encoding="utf8").readlines() ]
    words_from_books = re.findall(r'\w+', open('BOOKS.txt', read_mode, encoding="utf8").read())
    
    return words_from_dictionary + words_from_books

# El archivo se había abierto como escritura, pero se debe abrir en modo lectura.
WORDS = fetch_words('r') #<<<---
LETTERS = list(ascii_lowercase) + ['ñ', 'á', 'é', 'í', 'ó', 'ú']
SYMBOLS = ',.-¡!¿?#$@&%`´'
OPENSYMBOL = "¡¿`"
CLOSESYMBOL = "!?´"
OTHERSYMBOL = ',.-#$@&%`´'
# Definimos un índice de palabras en el cual guardamos la frecuencia con la que se utiliza en nuestros
# textos de referencia
WORDS_INDEX = {}

# Las ordenes estaban mal y/o al reves.
for word in WORDS:
    if word in WORDS_INDEX:
        WORDS_INDEX.update({word:WORDS_INDEX[word]+ 1})
    else:
        WORDS_INDEX[word] = 1

for symbol in SYMBOLS:
    WORDS_INDEX[symbol]=1
    
def possible_corrections(word):
    if filter_real_numbers(word):
        return filter_real_numbers(word)
    cap=word[0].isupper()
    lword=word.lower()
    single_word_possible_corrections = filter_real_words([lword])
    one_length_edit_possible_corrections = filter_real_words(one_length_edit(lword))
    two_lenght_edit_possible_corrections = filter_real_words(two_lenght_edit(lword))
    no_correction_at_all = word
    if single_word_possible_corrections:
        if cap:
            return {option.title() for option in single_word_possible_corrections}
        return single_word_possible_corrections
    elif one_length_edit_possible_corrections:
        if cap:
            return {option.title() for option in one_length_edit_possible_corrections}
        return one_length_edit_possible_corrections
    elif two_lenght_edit_possible_corrections:
        if cap:
            return {option.title() for option in two_lenght_edit_possible_corrections}
        return two_lenght_edit_possible_corrections
    else:
        return no_correction_at_all
    
def spell_check_sentence(sentence):
    stripped_sentence = correct_split(sentence)#list(map(lambda x : x, lower_cased_sentence.split()))
    checked = map(spell_check_word, stripped_sentence) #<<-- Ojo que cambié ese filter por map
    checked_words=list(checked)
    return correct_join(checked_words)#' '.join(checked_words)

def spell_check_word(word):
    cap=word[0].isupper()
    options=possible_corrections(word.lower())
    res=max(options, key=language_model)
    if word.replace("a","á") in options:
        return word.replace("a","á")
    if word.replace("e","é") in options:
        return word.replace("e","é")
    if word.replace("i","í") in options:
        return word.replace("i","í")
    if word.replace("o","ó") in options:
        return word.replace("o","ó")
    if word.replace("u","ú") in options:
        return word.replace("u","ú")
    if not cap:
        return res
    else:
        return res.title()

#def spell_check_word(word):
#    cap=word[0].isupper()
#    if not cap:
#        return max(possible_corrections(word.lower()), key=language_model)
#    else:
#        return max(possible_corrections(word.lower()), key=language_model).title()
    
def language_model(word):
    return WORDS_INDEX.get(word, 0)

def filter_real_words(words):
    return set(word for word in words if word.lower() in WORDS_INDEX)

def filter_real_numbers(word):
    try:
        temp=int(word)
        return set([word])
    except:
        return set()

def one_length_edit(word):
    '''Función no alterda por el ataque'''
    
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    #print(splits)
    removals_of_one_letter = []
    
    for left, right in splits:
        if right:
            removals_of_one_letter.append(left + right[1:])
            
    two_letters_transposes = []
    
    for left, right in splits:
        if len(right) > 1:
            two_letters_transposes.append(left + right[1] + right[0] + right[2:])
            
    one_letter_replaces = []
    
    for left, right in splits:
        if right:
            for c in LETTERS:
                one_letter_replaces.append(left + c + right[1:])
                
    one_letter_insertions = []
    
    for left, right in splits:
        for c in LETTERS:
            one_letter_insertions.append(left + c + right)
    
    one_length_editions = removals_of_one_letter + two_letters_transposes + one_letter_replaces + one_letter_insertions
    
    return list(set(one_length_editions))

def correct_split(sentence):
    """ Esta función crea una lista que divide la frase ubicando los signos aparte
        de las palabras
        
        Parametros
        ----------
        sentence: str
                La frase que se desea partir
    """
    stripped = list(map(lambda x : x, sentence.split()))
    stripped_sentence=[]
    for word in stripped:
        if any(car in word for car in SYMBOLS):
            stripped_sentence.extend(word_split(word))
        else:
            stripped_sentence.append(word)
    return stripped_sentence

def word_split(word):
    punt=SYMBOLS
    if not any(p in word for p in punt):
        return [word]
    tw=word
    carry=[]
    while(len(punt)>0):
        if(tw.find(punt[0])!=-1):
            ri=tw[:tw.find(punt[0])]
            c=tw[tw.find(punt[0])]
            rd=tw[tw.find(punt[0])+1:]
            if(len(ri)!=0):
                carry.extend(word_split(ri))
            carry.append(c)
            if(len(rd)!=0):
                carry.extend(word_split(rd))
            break
        punt=punt[1:]
    return carry

def correct_join(words):
    """ Esta función crea la frase con el formato correcto de las palabras
    
        Parametro
        ---------
        words: list
             Las palabras que se desean conectar, puede contener signos de puntuación
    """
    sentence=""
    for i in range(len(words)):
        if any(car in words[i] for car in OPENSYMBOL):
            sentence+=words[i]
        elif any(car in words[i] for car in CLOSESYMBOL):
            if len(sentence)>0:
                sentence=sentence[:-1]+words[i]
        elif not any(car in words[i] for car in OTHERSYMBOL):
            sentence+=words[i]
            if i<len(words)-1:
                sentence+=" "
        elif any(car in words[i] for car in OTHERSYMBOL):
            if len(sentence)!=0:
                if any(car in words[i-1] for car in CLOSESYMBOL):
                    sentence+=words[i]
                else:
                    sentence=sentence[:-1]+words[i]
                if i<len(words)-1:
                    sentence+=" "
    return sentence

def two_lenght_edit(word):
    '''Función no alterda por el ataque'''
    return [e2 for e1 in one_length_edit(word) for e2 in one_length_edit(e1)]

def test_spell_check_sentence():

    sentence = 'fabor guardar cilencio para no molestar'
    print(spell_check_sentence(sentence))
    assert 'favor guardar silencio para no molestar' == spell_check_sentence(sentence) 

    sentence = 'un lgar para la hopinion'
    print(spell_check_sentence(sentence))
    assert 'un lugar para la opinión' == spell_check_sentence(sentence)

    sentence = 'el Arebol del día'
    print(spell_check_sentence(sentence))
    assert 'el Arrebol del día' == spell_check_sentence(sentence)

    sentence = 'Rezpeto por la educasión'
    print(spell_check_sentence(sentence))
    assert 'Respeto por la educación' == spell_check_sentence(sentence)

    sentence = 'RTe encanta conduzir'
    print(spell_check_sentence(sentence))
    assert 'Te encanta conducir' == spell_check_sentence(sentence)

    sentence = 'HOy ay karne azada frezca siga pa dentro'
    print(spell_check_sentence(sentence))
    assert 'Hoy ay carne azada fresca siga la dentro' == spell_check_sentence(sentence)

    sentence = 'En mi ezcuela no enseñan a escrivir ni a ler'
    print(spell_check_sentence(sentence))
    assert 'En mi escuela no enseñan a escribir ni a le' == spell_check_sentence(sentence)

    sentence = 'él no era una persona de fiar pues era un mentirozo'
    print(spell_check_sentence(sentence))
    assert 'él no era una persona de fiar pues era un mentiroso' == spell_check_sentence(sentence)
    
    sentence = '%Él, no era una persona de fiar, ¡pues era un mentirozo!, lo sabe el 30% de la gente'
    print(spell_check_sentence(sentence))
    assert 'Él, no era una persona de fiar, ¡pues era un mentiroso!, lo saber el 30% de la gente' == spell_check_sentence(sentence)
test_spell_check_sentence()

favor guardar silencio para no molestar
un lugar para la opinión
el Arrebol del día
Respeto por la educación
Te encanta conducir
Hoy ay carne azada fresca siga la dentro
En mi escuela no enseñan a escribir ni a le
él no era una persona de fiar pues era un mentiroso
Él, no era una persona de fiar, ¡pues era un mentiroso!, lo saber el 30% de la gente


In [3]:
spell_check_sentence("hopinion")

'opinión'

In [5]:
WORDS_INDEX["el"]

440502